In [146]:
import pandas as pd
import numpy as np
import tensorflow as tf
from os import listdir
from sklearn.model_selection import train_test_split
print ('The version of TensorFlow is {}'.format(tf.__version__))

The version of TensorFlow is 1.4.0


In [147]:
root_path = './data'

appended_data = []
for file_name in listdir(root_path):
    file_path = root_path + '/' + file_name.encode().decode('utf-8')
    data_from_one_csv = pd.read_csv(file_path, skiprows=1)
    appended_data.append(data_from_one_csv)
    
data = pd.concat(appended_data, axis=0)
data = data.drop(columns = ['ACN', 'Date', 'Local Time Of Day', 'Ceiling', 'Callback', 'Callback.1', 'Unnamed: 96'])
data = data.rename(index=str, columns={"Flight Phase": "Flight Phase1"})

## drop the rows with empty synopsis description
data = data[pd.notnull(data['Synopsis'])]

X = data.drop(columns = 'Result')
Y_raw = pd.DataFrame(data['Result'])

processed_Y = []
for index, row in Y_raw.iterrows():
    #print (index, row['Result'])
    outcome = row['Result']
    if type(outcome) == np.float:
        res = 'unknown'
        processed_Y.append(res)
    elif ';' in outcome:
        res = str(outcome).split(';')[0]
        processed_Y.append(res)
    else:
        res = outcome
        processed_Y.append(res)

Y = pd.DataFrame(processed_Y, columns = ['Result'])

In [148]:
## compress the number of labels to be predicted --> map result to risk level
rate_nine = ['General Declared Emergency', 'General Physical Injury / Incapacitation', 'Flight Crew Inflight Shutdown', 
             'Air Traffic Control Separated Traffic', 'Aircraft Aircraft Damaged']

rate_seven = ['General Evacuated', 'Flight Crew Landed as Precaution', 'Flight Crew Regained Aircraft Control', 
              'Air Traffic Control Issued Advisory / Alert', 'Flight Crew Landed in Emergency Condition',
              'Flight Crew Landed In Emergency Condition']

rate_five = ['General Work Refused', 'Flight Crew Became Reoriented', 'Flight Crew Diverted', 
             'Flight Crew Executed Go Around / Missed Approach', 
             'Flight Crew Overcame Equipment Problem', 'Flight Crew Rejected Takeoff', 'Flight Crew Took Evasive Action', 
             'Air Traffic Control Issued New Clearance']

rate_three = ['General Maintenance Action', 'General Flight Cancelled / Delayed', 'General Release Refused / Aircraft Not Accepted', 
              'Flight Crew Overrode Automation', 'Flight Crew FLC Overrode Automation',
              'Flight Crew Exited Penetrated Airspace', 
              'Flight Crew Requested ATC Assistance / Clarification', 'Flight Crew Landed As Precaution',
              'Flight Crew Returned To Clearance', 'Flight Crew Returned To Departure Airport',
              'Aircraft Automation Overrode Flight Crew']

rate_one = ['General Police / Security Involved', 'Flight Crew Returned To Gate', 'Aircraft Equipment Problem Dissipated', 
            'unknown', 'Air Traffic Control Provided Assistance',
            'General None Reported / Taken', 'Flight Crew FLC complied w / Automation / Advisory']

Y_ = []
for i in range(Y.shape[0]):
    if Y['Result'][i] in rate_nine:
        Y_.append(5)
    elif Y['Result'][i] in rate_seven:
        Y_.append(4)
    elif Y['Result'][i] in rate_five:
        Y_.append(3)
    elif Y['Result'][i] in rate_three:
        Y_.append(2)
    elif Y['Result'][i] in rate_one:
        Y_.append(1)
    else:
        print (Y['Result'][i])

outcomes = np.asarray(Y_)
Y_pred = pd.DataFrame(Y_, index = X.index, columns = ['Result'])
unique, counts = np.unique(outcomes, return_counts=True)

In [149]:
data_rev = X.copy(deep=True)
data_rev['Result'] = Y_pred

## Upsampling the minority categories

In [150]:
from sklearn.utils import resample

df_majority_1 = data_rev[data_rev['Result']==1]
df_majority_3 = data_rev[data_rev['Result']==3]
df_minority_2 = data_rev[data_rev['Result']==2]
df_minority_4 = data_rev[data_rev['Result']==4]
df_minority_5 = data_rev[data_rev['Result']==5]

# Upsample minority class
df_minority_2_upsampled = resample(df_minority_2, 
                                 replace=True,     # sample with replacement
                                 n_samples=20000,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_4_upsampled = resample(df_minority_4, 
                                 replace=True,     # sample with replacement
                                 n_samples=20000,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_5_upsampled = resample(df_minority_5, 
                                 replace=True,     # sample with replacement
                                 n_samples=20000,    # to match majority class
                                 random_state=123) # reproducible results

df_upsampled = pd.concat([df_majority_1, df_majority_3, df_minority_2_upsampled, df_minority_4_upsampled, 
                          df_minority_5_upsampled])

df_upsampled['Result'].value_counts()

X = df_upsampled.drop(columns = 'Result')
Y_pred = df_upsampled['Result']

unique, counts = np.unique(Y_pred, return_counts=True)
print ('After the upsampling, the number of each item is: \n')
print (unique)
print (counts)

After the upsampling, the number of each item is: 

[1 2 3 4 5]
[20985 20000 20848 20000 20000]


## Split the data

In [232]:
from sklearn.model_selection import train_test_split

test_size_ratio = 0.2
random_split_seed = 100
X_train, X_test, Y_train, Y_test = train_test_split(X['Synopsis'], Y_pred, test_size = test_size_ratio, 
                                                    random_state = random_split_seed)
Y_test.head()

2601    3
740     1
378     4
372     3
1923    4
Name: Result, dtype: int64

## Pipeline: Naive Bayes

In [233]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB(alpha = 1, fit_prior=True)),
                    ])

text_clf.fit(X_train, Y_train)
pred_label = text_clf.predict(X_test)

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 5+1)]
print(classification_report(Y_test, pred_label, target_names=target_names))

             precision    recall  f1-score   support

          1       0.53      0.46      0.49      4206
          2       0.55      0.63      0.59      3937
          3       0.46      0.40      0.43      4174
          4       0.53      0.56      0.54      4032
          5       0.63      0.70      0.66      4018

avg / total       0.54      0.55      0.54     20367



## Model 1: Support Vector Machine with Linear Kernel

In [234]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='epsilon_insensitive', penalty='l2',
                                            alpha=1e-5, random_state=40,
                                            max_iter=10, tol=None)),
                    ])


parameters = {'clf__loss': ['epsilon_insensitive', 'hinge', 'log', 'huber', 'modified_huber', 'perceptron', 
                            'squared_loss', 'squared_epsilon_insensitive', 'squared_hinge'],
              'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3, 1e-4, 1e-5),
              'clf__penalty': ['l1', 'l2', 'elasticnet'],
              'clf__max_iter': (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
 }

optimal_parameters = {'clf__loss': ['modified_huber'],
              'vect__ngram_range':  [(1, 2)],
              'tfidf__use_idf': [True],
              'clf__alpha': [1e-5],
              'clf__penalty': ['elasticnet'],
              'clf__max_iter': [80],
 }

gs_clf = GridSearchCV(text_clf, optimal_parameters, n_jobs=-1)

gs_clf.fit(X_train, Y_train)
pred_label = gs_clf.predict(X_test)

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 6)]
print(classification_report(Y_test, pred_label, target_names=target_names))

             precision    recall  f1-score   support

          1       0.69      0.50      0.58      4206
          2       0.80      0.91      0.85      3937
          3       0.63      0.55      0.59      4174
          4       0.78      0.91      0.84      4032
          5       0.88      0.98      0.93      4018

avg / total       0.75      0.77      0.75     20367



In [235]:
print ('Accuracy: ', np.sum(np.equal(Y_test, pred_label).astype(int))/20367)
print ('The best set of parameters is \n', gs_clf.best_params_)

Accuracy:  0.765503019590514
The best set of parameters is 
 {'clf__alpha': 1e-05, 'clf__loss': 'modified_huber', 'clf__max_iter': 80, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


## Model 2: Processing other categorical data

In [236]:
## change column names
new_col_name = []
for col in X.columns:
    #print(type(col))
    new_col_name.append(col.replace('/ ', '').replace(' ', '_'))
    
X.columns = new_col_name

## output the headers from the csv file
X.keys()

data_type = []
for item_name in X.keys():
    data_type.append(type(X[item_name][0]))

print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('The unique data types across all the items are:', set(data_type))
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

for item_name in X.keys():
    ## find the number of NaN in this item
    no = np.sum(X[item_name].isna().astype(int))
    #print ('The number of {} with value equal to NaN is {}'.format(item_name, no))
    
    ## Replace the missing value with corresponding values
    if no > 0:
        if type(X[item_name][0]) == np.float64:
            X[item_name].fillna(-1, inplace = True)
        else:
            X[item_name].fillna('unknown', inplace = True)
X['Crew_Size'].head()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The unique data types across all the items are: {<class 'numpy.float64'>, <class 'str'>}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


0     2.0
8     2.0
13    2.0
14    2.0
15    2.0
Name: Crew_Size, dtype: float64

In [239]:
## Location
Locale_Reference = tf.feature_column.categorical_column_with_hash_bucket('Locale_Reference', 
                                                                         hash_bucket_size = len(set(X['Locale_Reference'])))
State_Reference = tf.feature_column.categorical_column_with_hash_bucket('State_Reference', 
                                                                        hash_bucket_size = len(set(X['State_Reference'])))


## Environment
Flight_Conditions = tf.feature_column.categorical_column_with_hash_bucket('Flight_Conditions', 
                                                                hash_bucket_size = len(set(X['State_Reference'])))
Weather_Elements_Visibility = tf.feature_column.categorical_column_with_hash_bucket('Weather_Elements_Visibility', 
                                                            hash_bucket_size = len(set(X['Weather_Elements_Visibility'])))
Work_Environment_Factor = tf.feature_column.categorical_column_with_hash_bucket('Work_Environment_Factor', 
                                                            hash_bucket_size = len(set(X['Work_Environment_Factor'])))
Light = tf.feature_column.categorical_column_with_hash_bucket('Light', hash_bucket_size = 
                                                              len(set(X['Work_Environment_Factor'])))


## Aircraft
ATC_Advisory = tf.feature_column.categorical_column_with_hash_bucket('ATC_Advisory', 
                                                            hash_bucket_size = len(set(X['ATC_Advisory'])))
Aircraft_Operator = tf.feature_column.categorical_column_with_hash_bucket('Aircraft_Operator', 
                                                                hash_bucket_size = len(set(X['Aircraft_Operator'])))
Make_Model_Name = tf.feature_column.categorical_column_with_hash_bucket('Make_Model_Name', 
                                                            hash_bucket_size = len(set(X['Make_Model_Name'])))
Crew_Size = tf.feature_column.numeric_column('Crew_Size', [1])
Flight_Plan = tf.feature_column.categorical_column_with_hash_bucket('Flight_Plan', 
                                                            hash_bucket_size = len(set(X['Flight_Plan'])))
Mission = tf.feature_column.categorical_column_with_hash_bucket('Mission', 
                                                                hash_bucket_size = len(set(X['Mission'])))
Flight_Phase1 = tf.feature_column.categorical_column_with_hash_bucket('Flight_Phase1', 
                                                                      hash_bucket_size = len(set(X['Flight_Phase1'])))
Route_In_Use = tf.feature_column.categorical_column_with_hash_bucket('Route_In_Use', 
                                                                     hash_bucket_size = len(set(X['Route_In_Use'])))
Airspace = tf.feature_column.categorical_column_with_hash_bucket('Airspace', 
                                                                 hash_bucket_size = len(set(X['Airspace'])))

## Component
Aircraft_Component = tf.feature_column.categorical_column_with_hash_bucket('Aircraft_Component', 
                                                             hash_bucket_size = len(set(X['Aircraft_Component'])))
Manufacturer = tf.feature_column.categorical_column_with_hash_bucket('Manufacturer', 
                                                        hash_bucket_size = len(set(X['Manufacturer'])))

## Person
Location_Of_Person = tf.feature_column.categorical_column_with_hash_bucket('Location_Of_Person', 
                                                                hash_bucket_size = len(set(X['Location_Of_Person'])))
Location_In_Aircraft = tf.feature_column.categorical_column_with_hash_bucket('Location_In_Aircraft',
                                                            hash_bucket_size = len(set(X['Location_In_Aircraft'])))
Reporter_Organization = tf.feature_column.categorical_column_with_hash_bucket('Reporter_Organization',
                                                            hash_bucket_size = len(set(X['Reporter_Organization'])))
Function = tf.feature_column.categorical_column_with_hash_bucket('Function', hash_bucket_size = len(set(X['Function'])))
Qualification = tf.feature_column.categorical_column_with_hash_bucket('Qualification', 
                                                                      hash_bucket_size = len(set(X['Qualification'])))
Human_Factors = tf.feature_column.categorical_column_with_hash_bucket('Human_Factors', 
                                                                      hash_bucket_size = len(set(X['Human_Factors'])))

## Events
Anomaly = tf.feature_column.categorical_column_with_hash_bucket('Anomaly', 
                                                                hash_bucket_size = len(set(X['Anomaly'])))
Detector = tf.feature_column.categorical_column_with_hash_bucket('Detector', 
                                                                 hash_bucket_size = len(set(X['Detector'])))
When_Detected = tf.feature_column.categorical_column_with_hash_bucket('When_Detected', 
                                                                      hash_bucket_size = len(set(X['When_Detected'])))
Were_Passengers_Involved_In_Event = tf.feature_column.categorical_column_with_hash_bucket('Were_Passengers_Involved_In_Event',
                                                    hash_bucket_size = len(set(X['Were_Passengers_Involved_In_Event'])))

## Assessments
Contributing_Factors_Situations = tf.feature_column.categorical_column_with_hash_bucket('Contributing_Factors_Situations', 
                                                   hash_bucket_size = len(set(X['Contributing_Factors_Situations'])))
Primary_Problem = tf.feature_column.categorical_column_with_hash_bucket('Primary_Problem', 
                                                        hash_bucket_size = len(set(X['Primary_Problem'])))

## Place
Locale_Reference = tf.feature_column.embedding_column(Locale_Reference, len(set(X['Locale_Reference'])))
State_Reference = tf.feature_column.embedding_column(State_Reference, len(set(X['State_Reference'])))


## Environment
Flight_Conditions = tf.feature_column.embedding_column(Flight_Conditions,  len(set(X['Flight_Conditions'])))
Weather_Elements_Visibility = tf.feature_column.embedding_column(Weather_Elements_Visibility,  
                                                                 len(set(X['Weather_Elements_Visibility'])))
Work_Environment_Factor = tf.feature_column.embedding_column(Work_Environment_Factor,  len(set(X['Work_Environment_Factor'])))
Light = tf.feature_column.embedding_column(Light, len(set(X['Light'])))


## Aircraft
ATC_Advisory = tf.feature_column.embedding_column(ATC_Advisory, len(set(X['ATC_Advisory'])))
Aircraft_Operator = tf.feature_column.embedding_column(Aircraft_Operator, len(set(X['Aircraft_Operator'])))
Make_Model_Name = tf.feature_column.embedding_column(Make_Model_Name, len(set(X['Make_Model_Name'])))
Flight_Plan = tf.feature_column.embedding_column(Flight_Plan, len(set(X['Flight_Plan'])))
Mission = tf.feature_column.embedding_column(Mission, len(set(X['Mission'])))
Flight_Phase1 = tf.feature_column.embedding_column(Flight_Phase1, len(set(X['Flight_Phase1'])))
Route_In_Use = tf.feature_column.embedding_column(Route_In_Use, len(set(X['Route_In_Use'])))
Airspace = tf.feature_column.embedding_column(Airspace, len(set(X['Airspace'])))

## Component
Aircraft_Component = tf.feature_column.embedding_column(Aircraft_Component, len(set(X['Aircraft_Component'])))
Manufacturer = tf.feature_column.embedding_column(Manufacturer, len(set(X['Manufacturer'])))

## Person
Location_Of_Person = tf.feature_column.embedding_column(Location_Of_Person, len(set(X['Location_Of_Person'])))
Location_In_Aircraft = tf.feature_column.embedding_column(Location_In_Aircraft, len(set(X['Location_In_Aircraft'])))
Reporter_Organization = tf.feature_column.embedding_column(Reporter_Organization, len(set(X['Reporter_Organization'])))
Function = tf.feature_column.embedding_column(Function, len(set(X['Function'])))
Qualification = tf.feature_column.embedding_column(Qualification, len(set(X['Qualification'])))
Human_Factors = tf.feature_column.embedding_column(Human_Factors, len(set(X['Human_Factors'])))

## Events
Anomaly = tf.feature_column.embedding_column(Anomaly, len(set(X['Anomaly'])))
Detector = tf.feature_column.embedding_column(Detector, len(set(X['Detector'])))
When_Detected = tf.feature_column.embedding_column(When_Detected, len(set(X['When_Detected'])))
Were_Passengers_Involved_In_Event = tf.feature_column.embedding_column(Were_Passengers_Involved_In_Event,
                                                                       len(set(X['Were_Passengers_Involved_In_Event'])))

## Assessments
Contributing_Factors_Situations = tf.feature_column.embedding_column(Contributing_Factors_Situations,
                                                                     len(set(X['Contributing_Factors_Situations'])))
Primary_Problem = tf.feature_column.embedding_column(Primary_Problem, len(set(X['Primary_Problem'])))

In [240]:
from sklearn.model_selection import train_test_split

X_sub = X[['Locale_Reference', 'State_Reference', 'Flight_Conditions', 'Weather_Elements_Visibility', 
            'Work_Environment_Factor', 'Light', 'ATC_Advisory', 'Aircraft_Operator', 'Make_Model_Name', 
            'Crew_Size', 'Flight_Plan', 'Mission', 'Flight_Phase1',
            'Route_In_Use','Airspace', 'Aircraft_Component', 'Manufacturer', 'Location_Of_Person', 'Location_In_Aircraft',
            'Reporter_Organization', 'Function', 'Qualification', 'Human_Factors', 'Anomaly', 'Detector', 'When_Detected',
            'Were_Passengers_Involved_In_Event', 'Contributing_Factors_Situations', 'Primary_Problem']]

X_train, X_test, Y_train, Y_test = train_test_split(X_sub, Y_pred, test_size = test_size_ratio, 
                                                    random_state = random_split_seed)

In [155]:
label = []
number_models = 5
for i in range(number_models):
    print ('Train the {} model, please keep waiting !!!'.format(i+1))
    print ('\n')
    
    X_train_set, X_test_tmp, Y_train_set, Y_test_tmp = train_test_split(X_train, Y_train, test_size = 0.2, random_state = i)

    ## define input function
    input_func = tf.estimator.inputs.pandas_input_fn(x = X_train_set, y = Y_train_set, batch_size = 500, 
                                                        num_epochs = 600, shuffle = True)

    ## define the feature columns
    feat_cols = [Locale_Reference, State_Reference, Flight_Conditions, Weather_Elements_Visibility, Work_Environment_Factor, 
                     Light, ATC_Advisory, Aircraft_Operator, Make_Model_Name, Crew_Size, Flight_Plan, Mission, Flight_Phase1, 
                     Route_In_Use, Airspace, Aircraft_Component, Manufacturer, Location_Of_Person, Location_In_Aircraft, 
                     Reporter_Organization, Function, Qualification, Human_Factors, Anomaly, Detector, When_Detected, 
                     Were_Passengers_Involved_In_Event, Contributing_Factors_Situations, Primary_Problem]

    ## build the model
    model = tf.estimator.DNNClassifier(hidden_units = [40, 40, 40, 40, 40, 40, 40, 40], feature_columns = feat_cols,
                                           n_classes = 6, optimizer = tf.train.AdamOptimizer(learning_rate = 0.001))
        
    ## train the model
    model.train(input_fn = input_func, steps = 4000)
    
    
    ## make predictions
    eval_input = tf.estimator.inputs.pandas_input_fn(x = X_test, shuffle = False)
    prediction = list(model.predict(eval_input))

    pred_label = [int(pred['class_ids']) for pred in prediction]
    
    label.append(pred_label)
    
    
ensembel_pred = []
for j in range(len(label[0])):
    x = np.zeros(shape = (len(label), 1)) - 1
    for i in range(len(label)):
        x[i] =  label[i][j]
    (values, counts) = np.unique(x, return_counts=True)
    ind = np.argmax(counts)
    ensembel_pred.append((values[ind]))

ensembel_pred

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 5+1)]
print(classification_report(Y_test, ensembel_pred, target_names=target_names))

Train the 1 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmpustpcu7m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000218C6698C18>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpustpcu7m\model.ckpt.
INFO:tensorflow:loss = 895.7074, step = 1
INFO:tensorflow:global_step/sec: 0.647053
INFO:tensorflow:loss = 631.00024, step = 101 (154.589 sec)
INFO:tensorflow:global_step/sec: 0.65106

INFO:tensorflow:loss = 251.03018, step = 801 (162.749 sec)
INFO:tensorflow:global_step/sec: 0.671068
INFO:tensorflow:loss = 228.25932, step = 901 (149.081 sec)
INFO:tensorflow:global_step/sec: 0.657448
INFO:tensorflow:loss = 232.71298, step = 1001 (152.075 sec)
INFO:tensorflow:global_step/sec: 0.650375
INFO:tensorflow:loss = 171.76004, step = 1101 (153.831 sec)
INFO:tensorflow:Saving checkpoints for 1169 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpfgibw1gu\model.ckpt.
INFO:tensorflow:global_step/sec: 0.599216
INFO:tensorflow:loss = 180.24825, step = 1201 (166.810 sec)
INFO:tensorflow:global_step/sec: 0.660215
INFO:tensorflow:loss = 148.27768, step = 1301 (151.465 sec)
INFO:tensorflow:global_step/sec: 0.655477
INFO:tensorflow:loss = 159.76556, step = 1401 (152.568 sec)
INFO:tensorflow:global_step/sec: 0.66589
INFO:tensorflow:loss = 166.51727, step = 1501 (150.172 sec)
INFO:tensorflow:Saving checkpoints for 1557 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpfgibw1gu\model.ckpt.
INFO:tens

INFO:tensorflow:Saving checkpoints for 2352 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpqafd1r4y\model.ckpt.
INFO:tensorflow:global_step/sec: 0.611429
INFO:tensorflow:loss = 105.84696, step = 2401 (163.560 sec)
INFO:tensorflow:global_step/sec: 0.670096
INFO:tensorflow:loss = 78.83223, step = 2501 (149.240 sec)
INFO:tensorflow:global_step/sec: 0.675007
INFO:tensorflow:loss = 91.49517, step = 2601 (148.138 sec)
INFO:tensorflow:global_step/sec: 0.673238
INFO:tensorflow:loss = 86.30971, step = 2701 (148.567 sec)
INFO:tensorflow:Saving checkpoints for 2746 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpqafd1r4y\model.ckpt.
INFO:tensorflow:global_step/sec: 0.604739
INFO:tensorflow:loss = 108.97676, step = 2801 (165.459 sec)
INFO:tensorflow:global_step/sec: 0.673066
INFO:tensorflow:loss = 81.02835, step = 2901 (148.450 sec)
INFO:tensorflow:global_step/sec: 0.67308
INFO:tensorflow:loss = 86.6936, step = 3001 (148.687 sec)
INFO:tensorflow:global_step/sec: 0.67174
INFO:tensorflow:loss = 73.07594,

INFO:tensorflow:global_step/sec: 0.67259
INFO:tensorflow:loss = 57.301857, step = 3901 (148.690 sec)
INFO:tensorflow:Saving checkpoints for 3937 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpd8lcey2f\model.ckpt.
INFO:tensorflow:Saving checkpoints for 4000 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpd8lcey2f\model.ckpt.
INFO:tensorflow:Loss for final step: 61.81904.
INFO:tensorflow:Restoring parameters from C:\Users\ZHANGX~1\AppData\Local\Temp\tmpd8lcey2f\model.ckpt-4000
Train the 5 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmplnfld5i5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0

## Hybrid model

In [250]:
len(ensembel_pred)

20367

In [276]:
label = []
for i in range(len(ensembel_pred)):
    if pred_label[i] == ensembel_pred[i]:
        label.append(pred_label[i])
    else:
        label.append(np.random.choice([pred_label[i], ensembel_pred[i]], p = [0.9, 0.1]))

In [277]:
print(classification_report(Y_test, label, target_names=target_names))

             precision    recall  f1-score   support

          1       0.68      0.51      0.58      4206
          2       0.80      0.91      0.85      3937
          3       0.62      0.54      0.58      4174
          4       0.78      0.91      0.84      4032
          5       0.88      0.97      0.93      4018

avg / total       0.75      0.76      0.75     20367



In [251]:
Y_test.head()

2601    3
740     1
378     4
372     3
1923    4
Name: Result, dtype: int64

In [283]:
pred_label

array([3, 2, 4, ..., 2, 2, 3], dtype=int64)